In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This script is used to compute negative control datasets.

In [ ]:
from IPython import display
import pandas as pd
import numpy as np
import numpy as np
import numpy.ma as ma
import scipy
import scipy.stats
from scipy.stats import gaussian_kde
from scipy.stats import t

from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from matplotlib import cm
from matplotlib.colors import Normalize 
from matplotlib.offsetbox import AnchoredText

from tensorflow import keras
from sklearn.neighbors import KernelDensity

from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import pickle
from timeit import default_timer as timer
Folder = '/content/drive/MyDrive/SCRaPL/'
Techn = '10X'

Load, permute and save data.

In [ ]:
if Techn == 'NMT':

      with open(Folder+'Demo/Data/yy_met_300_100_'+Techn+'.pickle','rb') as handle: 
          yy_met = pickle.load(handle)
      with open(Folder+'Demo/Data/yy_exp_300_100_'+Techn+'.pickle','rb') as handle: 
          yy_exp = pickle.load(handle)
      with open(Folder+'Demo/Data/yy_cpg_300_100_'+Techn+'.pickle','rb') as handle: 
          CpG = pickle.load(handle)
      with open(Folder+'Demo/Data/Norm_300_100_'+Techn+'.pickle','rb') as handle: 
          nrm = pickle.load(handle)

      N_ft = yy_exp.shape[0]
      N_cls = yy_exp.shape[1]

      nrm = np.repeat(nrm,repeats=N_ft,axis=0)

      yy_met = np.asarray(yy_met,dtype=np.float32)
      CpG = np.asarray(CpG,dtype=np.float32)
      yy_exp = np.asarray(yy_exp,dtype=np.float32)
      nrm = np.asarray(nrm,dtype=np.float32)

      yy_met_new = yy_met.copy()
      CpG_new = CpG.copy()
      yy_exp_new = yy_exp.copy()
      nrm_new = nrm.copy()

      perm_met_all = np.zeros((N_ft,N_cls))
      perm_rna_all = np.zeros((N_ft,N_cls))
      for ii in range(N_ft): 

          perm_met = np.random.permutation(N_cls)
          perm_rna = np.random.permutation(N_cls)

          yy_exp_new[ii,:] = yy_exp[ii,perm_rna]
          nrm_new[ii,:] = nrm[ii,perm_rna]

          yy_met_new[ii,:] = yy_met[ii,perm_met]
          CpG_new[ii,:] = CpG[ii,perm_met]

          perm_rna_all[ii,:] = perm_rna  
          perm_met_all[ii,:] = perm_met  

      neg_ind = 1

      with open(Folder+'Demo/Data/yy_met_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(yy_met_new, handle)

      with open(Folder+'Demo/Data/yy_cpg_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(CpG_new, handle)

      with open(Folder+'Demo/Data/yy_exp_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(yy_exp_new, handle)

      with open(Folder+'Demo/Data/Norm_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(nrm_new, handle)

      with open(Folder+'Demo/Data/Perm_met_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(perm_met_all, handle)

      with open(Folder+'Demo/Data/Perm_rna_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(perm_rna_all, handle)

elif Techn == '10X':

      with open(Folder+'Demo/Data/yy_acc_300_100_'+Techn+'.pickle','rb') as handle: 
          yy_acc = pickle.load(handle)
      with open(Folder+'Demo/Data/yy_exp_300_100_'+Techn+'.pickle','rb') as handle: 
          yy_exp = pickle.load(handle)
      with open(Folder+'Demo/Data/Norm_acc_300_100_'+Techn+'.pickle','rb') as handle: 
          nrm_acc = pickle.load(handle)
      with open(Folder+'Demo/Data/Norm_exp_300_100_'+Techn+'.pickle','rb') as handle: 
          nrm_exp = pickle.load(handle)

      N_ft = yy_exp.shape[0]
      N_cls = yy_exp.shape[1]

      nrm_acc = np.repeat(nrm_acc,repeats=N_ft,axis=0)
      nrm_exp = np.repeat(nrm_exp,repeats=N_ft,axis=0)

      yy_acc = np.asarray(yy_acc,dtype=np.float32)
      yy_exp = np.asarray(yy_exp,dtype=np.float32)
      
      nrm_acc = np.asarray(nrm_acc,dtype=np.float32)
      nrm_acc = np.asarray(nrm_acc,dtype=np.float32)

      yy_acc_new = yy_acc.copy()
      yy_exp_new = yy_exp.copy()
      nrm_acc_new = nrm_acc.copy()
      nrm_exp_new = nrm_exp.copy()

      perm_acc_all = np.zeros((N_ft,N_cls))
      perm_rna_all = np.zeros((N_ft,N_cls))

      for ii in range(N_ft): 

          perm_acc = np.random.permutation(N_cls)
          perm_rna = np.random.permutation(N_cls)

          yy_exp_new[ii,:] = yy_exp[ii,perm_rna]
          nrm_exp_new[ii,:] = nrm_exp[ii,perm_rna]

          yy_acc_new[ii,:] = yy_acc[ii,perm_acc]
          nrm_acc_new[ii,:] = nrm_acc[ii,perm_acc]

          perm_rna_all[ii,:] = perm_rna  
          perm_acc_all[ii,:] = perm_acc


      neg_ind = 1

      with open(Folder+'Demo/Data/yy_acc_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(yy_acc_new, handle)

      with open(Folder+'Demo/Data/Norm_acc_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(nrm_exp_new, handle)

      with open(Folder+'Demo/Data/yy_exp_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(yy_exp_new, handle)

      with open(Folder+'Demo/Data/Norm_exp_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(nrm_exp_new, handle)


      with open(Folder+'Demo/Data/Perm_acc_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(perm_acc_all, handle)

      with open(Folder+'Demo/Data/Perm_rna_300_100_'+Techn+'_neg'+str(neg_ind)+'.pickle', 'wb') as handle:
          pickle.dump(perm_rna_all, handle)
else:

      print('Please choose a correct input')
